In [45]:
import pandas as pd
import pickle

In [46]:
path = "dataset/"

outfit_data = pd.read_csv(path + "/outfit_data.csv")
product_data = pd.read_csv(path + "/product_data.csv")

In [47]:
non_interseting = product_data[product_data['des_product_family'].isin(['Fragances', 'Deco Accessories', 'Intimate', 'Deco Textiles', 'Bedding'])]

non_interseting_codes = non_interseting['cod_modelo_color']
outfit_data = outfit_data[~outfit_data['cod_modelo_color'].isin(non_interseting_codes)]

In [48]:
# outfit_data['cod_modelo_color'] = outfit_data['cod_modelo_color'].str.slice(stop=-3)
outfit_data = outfit_data.drop_duplicates()
product_counts = outfit_data.groupby('cod_modelo_color').size().reset_index(name='counts')

merged_df = pd.merge(outfit_data, outfit_data, how='left', on='cod_outfit')
merged_df = merged_df[merged_df['cod_modelo_color_x'] > merged_df['cod_modelo_color_y']]
merged_df = merged_df.groupby(['cod_modelo_color_x', 'cod_modelo_color_y']).size().reset_index(name='weight')

# normalize the weights by the number of times the product appears in an outfit
merged_df = pd.merge(merged_df, product_counts, how='left', left_on='cod_modelo_color_x', right_on='cod_modelo_color')
merged_df['weight'] = merged_df['weight'] / merged_df['counts']

merged_df['edge'] = merged_df.apply(lambda x: (x['cod_modelo_color_y'], x['weight']), axis=1)
merged_df = merged_df.groupby('cod_modelo_color_x').agg({'edge': lambda x: list(x)})
merged_df

,edge
cod_modelo_color_x,
41041002-02,"[(41039067-05, 0.5)]"
41045021-02,"[(41025020-02, 0.14285714285714285)]"
41045815-45,"[(41039067-05, 0.25)]"
41065822-50,"[(41055822-08, 1.0)]"
41083009-08,"[(41069066-05, 0.14285714285714285)]"
...,...
67091003-99,"[(57029209-99, 1.0), (57095923-99, 1.0), (6700..."
67106705-99,"[(57025921-OR, 1.0), (57037880-OR, 1.0), (5708..."
87040069-OR,"[(57000438-OR, 1.0), (57001503-02, 1.0), (5701..."


In [54]:
import networkx as nx

G = nx.Graph()

for row in merged_df.iterrows():
    node = row[0]
    for edge in row[1]['edge']:
        G.add_edge(node, edge[0], weight=edge[1])

In [55]:
communities = nx.community.louvain_communities(G)

In [59]:
# create a list of tuples with id and community

communities = list(communities)

data = []

for i, community in enumerate(communities):
    data += [(i, item) for item in community]

data

[(0, '41085754-09'),
 (0, '43003701-TO'),
 (0, '41045021-02'),
 (0, '53000928-37'),
 (0, '53040970-02'),
 (0, '51080896-07'),
 (0, '53090690-02'),
 (0, '43095026-87'),
 (0, '53030654-01'),
 (0, '51060897-TM'),
 (0, '53060568-30'),
 (0, '43045011-42'),
 (0, '51090918-43'),
 (0, '43035019-08'),
 (0, '53071008-37'),
 (0, '43027739-01'),
 (0, '51030705-05'),
 (0, '43003738-OR'),
 (0, '43075794-OR'),
 (0, '57022877-02'),
 (0, '51030736-20'),
 (0, '53080604-70'),
 (0, '53000588-TM'),
 (0, '51067019-69'),
 (0, '43095826-02'),
 (0, '57084753-08'),
 (0, '43057751-08'),
 (0, '51065743-99'),
 (0, '53060992-99'),
 (0, '53070721-05'),
 (0, '43083732-99'),
 (0, '53080586-81'),
 (0, '51070586-02'),
 (0, '43033720-52'),
 (0, '43069088-08'),
 (0, '51060676-15'),
 (0, '53050548-99'),
 (0, '51030704-05'),
 (0, '53090712-07'),
 (0, '57016305-37'),
 (0, '43025746-08'),
 (0, '51093024-99'),
 (0, '43089077-01'),
 (0, '51051013-01'),
 (0, '51000622-85'),
 (0, '53060600-05'),
 (0, '53083035-50'),
 (0, '5103073

In [56]:
communities = list(communities)
print(len(communities))

50


In [53]:
compatibilities = dict(zip(merged_df.cod_modelo_color_x, merged_df.cod_modelo_color_y))

with open(path + 'compatibilities.pickle', 'wb') as handle:
    pickle.dump(compatibilities, handle, protocol=pickle.HIGHEST_PROTOCOL)

AttributeError: 'DataFrame' object has no attribute 'cod_modelo_color_x'